In [1]:
from bs4 import BeautifulSoup as bs

In [2]:
import requests

In [3]:
import pprint

In [4]:
import pandas as pd

In [5]:
url = 'https://roscontrol.com/category/produkti/'

In [6]:
headers = {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36',
           'Accept': '*/*'}

In [7]:
cat_selection = requests.get(url, headers=headers)

In [8]:
soup = bs(cat_selection.text, 'html.parser')

In [9]:
cat_names = soup.find_all('div',attrs={'class':'catalog__category-name'})

In [10]:
for i in range(len(cat_names)):
    cat_names[i] = cat_names[i].string

In [11]:
cat_links = soup.find_all('a',attrs={'class':'catalog__category-item'})

In [12]:
for i in range(len(cat_links)):
    cat_links[i] = cat_links[i]['href'].replace('/category/produkti/', '')

In [13]:
categories = {
    "Наименование": cat_names,
    "Ссылка": cat_links
}

categories = pd.DataFrame(categories)

In [14]:
category = categories.loc[int(input(categories["Наименование"])), "Ссылка"]

0                       Молочные продукты 
1                         Мясные продукты 
2                                   Птица 
3                     Рыба и морепродукты 
4                     Вода, соки, напитки 
5                   Хлебобулочные изделия 
6                         Детское питание 
7                                    Яйца 
8                                 Бакалея 
9                         Горячие напитки 
10                   Кондитерские изделия 
11                    Алкогольные напитки 
12                                  Соусы 
13                          Полуфабрикаты 
14                     Растительное масло 
15                               Консервы 
16                                  Снеки 
17                   Фрукты, овощи, грибы 
18                              Маргарины 
19     Готовые блюда и кулинарные изделия 
Name: Наименование, dtype: object13


In [15]:
subcat_selection = requests.get(url + category, headers=headers)

In [16]:
soup = bs(subcat_selection.text, 'html.parser')

In [17]:
subcat_names = soup.find_all('div',attrs={'class':'catalog__category-name'})

if subcat_names:
    for i in range(len(subcat_names)):
        subcat_names[i] = subcat_names[i].string

    subcat_links = soup.find_all('a',attrs={'class':'catalog__category-item'})

    for i in range(len(subcat_links)):
        subcat_links[i] = subcat_links[i]['href'].replace('/category/produkti/' + category, '')

    subcategories = {
        "Наименование": subcat_names,
        "Ссылка": subcat_links
    }

    subcategories = pd.DataFrame(subcategories)
    subcategory = subcategories.loc[int(input(subcategories["Наименование"])), "Ссылка"]
else:
    subcategory = ''

0                Пельмени и хинкали 
1      Блюда быстрого приготовления 
2                          Наггетсы 
3                           Сырники 
4                          Блинчики 
5                           Котлеты 
6                             Пицца 
7                          Вареники 
8                             Тесто 
9              Полуфабрикаты мясные 
10            Полуфабрикаты из рыбы 
Name: Наименование, dtype: object1


In [18]:
response = requests.get(url + category + subcategory, headers=headers)

In [19]:
soup = bs(response.text, 'html.parser')

In [20]:
products = soup.find_all('div', attrs={'class':'wrap-product-catalog__item'})

In [21]:
products_list = []
for product in products:
    product_data = {}
    name = product.find('div', attrs={'class':'product__item-link'}).getText()
    link = 'https://roscontrol.com' + product.contents[1]['href']
    try:
        rating_status = product.find('div', attrs={'class':'rating-total-text'}).string
    except AttributeError:
        rating_status = False
    if rating_status == 'Общая оценка':
        total_rating = product.find('div', attrs={'class':'product-rating'}).contents[1].getText()
        rating_block = product.find('div', attrs={'class':'rating-block'})
        for row in rating_block.findChildren(recursive=False):
            parameter = row.findChildren(recursive=False)[0].findChildren(recursive=False)[0].getText()
            value = row.findChildren(recursive=False)[1].getText()
            product_data[parameter] = value
    elif rating_status == 'В чёрном списке':
        total_rating = 'В чёрном списке'
    elif not rating_status:
        total_rating = 'Нет информации о товаре'
    product_data['Наименование'] = name
    product_data['Ссылка'] = link
    product_data['Общий рейтинг'] = total_rating
    
    products_list.append(product_data)

In [22]:
pprint.pp(products_list, sort_dicts=False)

[{'Пищевая ценность': '30',
  'Натуральность': '33',
  'Безопасность': '92',
  'Качество': '25',
  'Наименование': 'Изделия макаронные быстрого приготовления "Роллтон"',
  'Ссылка': 'https://roscontrol.com/product/sup-kuriniy-rollton/',
  'Общий рейтинг': ' 45 '},
 {'Пищевая ценность': '16',
  'Натуральность': '28',
  'Безопасность': '77',
  'Качество': '50',
  'Наименование': 'Суп куриный "Приправыч"',
  'Ссылка': 'https://roscontrol.com/product/sup-kuriniy-pripravich/',
  'Общий рейтинг': ' 43 '},
 {'Пищевая ценность': '28',
  'Натуральность': '30',
  'Безопасность': '84',
  'Качество': '64',
  'Наименование': 'Суп куриный Podravka',
  'Ссылка': 'https://roscontrol.com/product/sup-kuriniy-podravka/',
  'Общий рейтинг': ' 52 '},
 {'Пищевая ценность': '3',
  'Натуральность': '0',
  'Безопасность': '71',
  'Качество': '45',
  'Наименование': 'Суп куриный Gallina Blanca',
  'Ссылка': 'https://roscontrol.com/product/sup-kuriniy-gallina-blanca/',
  'Общий рейтинг': ' 30 '},
 {'Пищевая ценн

In [23]:
df_products = pd.DataFrame(products_list)

In [24]:
df_products

,Пищевая ценность,Натуральность,Безопасность,Качество,Наименование,Ссылка,Общий рейтинг
0,30,33,92,25,"Изделия макаронные быстрого приготовления ""Рол...",https://roscontrol.com/product/sup-kuriniy-rol...,45
1,16,28,77,50,"Суп куриный ""Приправыч""",https://roscontrol.com/product/sup-kuriniy-pri...,43
2,28,30,84,64,Суп куриный Podravka,https://roscontrol.com/product/sup-kuriniy-pod...,52
3,3,0,71,45,Суп куриный Gallina Blanca,https://roscontrol.com/product/sup-kuriniy-gal...,30
4,19,12,70,49,"Суп куриный ""Бакалея 101""",https://roscontrol.com/product/sup-kuriniy-bak...,38
5,16,42,84,58,Суп куриный Maggi,https://roscontrol.com/product/sup-kuriniy-maggi/,50
6,0,23,85,49,Пюре картофельное Smart,https://roscontrol.com/product/pyure-kartofeln...,39
7,0,23,85,45,"Пюре картофельное ""Быстринка""",https://roscontrol.com/product/pyure-kartofeln...,38
8,1,23,80,47,"Пюре картофельное ""Каждый день""",https://roscontrol.com/product/pyure-kartofeln...,38
9,12,48,78,60,Пюре картофельное Big Bon,https://roscontrol.com/product/pyure-kartofeln...,50


In [26]:
df_products.to_csv('df_products.csv', index=)

In [27]:
pd.read_csv('df_products.csv')

,Unnamed: 0,Пищевая ценность,Натуральность,Безопасность,Качество,Наименование,Ссылка,Общий рейтинг
0,0,30.0,33.0,92.0,25.0,"Изделия макаронные быстрого приготовления ""Рол...",https://roscontrol.com/product/sup-kuriniy-rol...,45
1,1,16.0,28.0,77.0,50.0,"Суп куриный ""Приправыч""",https://roscontrol.com/product/sup-kuriniy-pri...,43
2,2,28.0,30.0,84.0,64.0,Суп куриный Podravka,https://roscontrol.com/product/sup-kuriniy-pod...,52
3,3,3.0,0.0,71.0,45.0,Суп куриный Gallina Blanca,https://roscontrol.com/product/sup-kuriniy-gal...,30
4,4,19.0,12.0,70.0,49.0,"Суп куриный ""Бакалея 101""",https://roscontrol.com/product/sup-kuriniy-bak...,38
5,5,16.0,42.0,84.0,58.0,Суп куриный Maggi,https://roscontrol.com/product/sup-kuriniy-maggi/,50
6,6,0.0,23.0,85.0,49.0,Пюре картофельное Smart,https://roscontrol.com/product/pyure-kartofeln...,39
7,7,0.0,23.0,85.0,45.0,"Пюре картофельное ""Быстринка""",https://roscontrol.com/product/pyure-kartofeln...,38
8,8,1.0,23.0,80.0,47.0,"Пюре картофельное ""Каждый день""",https://roscontrol.com/product/pyure-kartofeln...,38
9,9,12.0,48.0,78.0,60.0,Пюре картофельное Big Bon,https://roscontrol.com/product/pyure-kartofeln...,50
